In [ ]:
import datetime as dt
import time
import ib_insync
import asyncio
import pandas as pd
print(ib_insync.__all__)
from ib_insync import *
util.startLoop()

In [ ]:
ib = IB()
ib.connect('127.0.0.1', 7497, clientId=33)

contracts = [Forex(pair) for pair in ('AUDUSD', 'EURCHF', 'GBPUSD', 'EURGBP', 'EURUSD', 'USDCHF')]
ib.qualifyContracts(*contracts)
for contract in contracts:
    ib.reqMktData(contract, '', False, False)   

In [ ]:
global short
global long
global emaG
global price
global action
global update
global firstT
global ticker
global sym
global cur
global tik
global openprice
global sl
global slopenprice
sym = "USD"
cur = "CHF"
tik = sym+cur
short = False
long = False
emaG = 0
price = 0
action = "DONOTHING"
update = False
firstT = True
openprice = 0
sl = False
slopenprice = 0

In [ ]:
def checkPositions(symbol,currency,positions):
    notOpen = True
    for x in positions:
        stock = x[1]
        if(stock.symbol == symbol and stock.currency == currency):
            notOpen = False
    return notOpen

def checkSell(symbol,currency,positions):
    sell = True
    for x in positions:
        stock = x[1]
        price = x[2]
        if(stock.symbol == symbol and stock.currency == currency):
            if(price < 0):
                sell = True
            elif(price > 0):
                sell = False
            return sell
    
def checkOpenSell(symbol,openBid,closeBid,ema,currency,positions):
    openSell = False
    if(openBid < ema and closeBid > ema and checkPositions(symbol,currency,positions)):
        openSell = True
    return openSell
    
def checkOpenBuy(symbol,openBid,closeBid,ema,currency,positions):
    openBuy = False
    if(openBid > ema and closeBid < ema and checkPositions(symbol,currency,positions)):
        openSell = True
    return openBuy

def checkCloseSell(symbol,closeBid,ema,currency,positions):
    closeSell = False
    if(not checkPositions(symbol,currency,positions) and checkSell(symbol,currency,positions) and closeBid < ema):
        closeSell = True
    return closeSell

def checkCloseBuy(symbol,closeBid,ema,currency,positions):
    closeBuy = False
    if(not checkPositions(symbol,currency,positions) and not checkSell(symbol,currency,positions) and closeBid > ema):
        closeBuy = True
    return closeBuy

                  
def stopLoss(oprice,cprice):
    global action
    global short
    global long
    global tik
    global openprice
    global short
    global long
    global slopenprice
    global sl
    stoppoints = 0.0015
    contract = Forex(tik)
    if(oprice != 0 and not sl):
        if(cprice >= oprice + stoppoints and short):
            order = MarketOrder('BUY', 200000)
            ib.placeOrder(contract, order)
            action = "DONOTHING"
            short = False
            long = True
            slopenprice = cprice
            openprice = 0
            sl = True
        elif(cprice <= oprice - stoppoints and long):
            order = MarketOrder('SELL', 200000)
            ib.placeOrder(contract, order)
            action = "DONOTHING"
            short = True
            long = False
            slopenprice = cprice
            openprice = 0
            sl = True

def stopLossExt(cprice):
    stoppoints = 0.0015
    global slopenprice
    global sl
    global short
    global long
    global tik
    global openprice
    contract = Forex(tik)
    if(sl):
        if(cprice >= slopenprice + stoppoints and long):
            order = MarketOrder('SELL', 100000)
            ib.placeOrder(contract, order)
            action = "DONOTHING"
            short = False
            long = False
            slopenprice = 0
            sl = False
            openprice = 0
        elif(cprice <= slopenprice - stoppoints and short):
            order = MarketOrder('BUY', 100000)
            ib.placeOrder(contract, order)
            action = "DONOTHING"
            short = False
            long = False
            slopenprice = 0
            sl = False
            openprice = 0
        elif(cprice >=slopenprice + stoppoints and short):
            order = MarketOrder('BUY', 100000)
            ib.placeOrder(contract, order)
            action = "DONOTHING"
            short = False
            long = False
            slopenprice = 0
            sl = False
            openprice = 0
        elif(cprice <= slopenprice - stoppoints and long):
            order = MarketOrder('BUY', 100000)
            ib.placeOrder(contract, order)
            action = "DONOTHING"
            short = False
            long = False
            slopenprice = 0
            sl = False
            openprice = 0
            

In [ ]:
#run every minute

def getAction(symbol,currency):
    global emaG
    global action
    contract = Contract()
    contract.symbol = symbol #Left
    contract.secType = "CASH"
    contract.currency = currency #Right
    contract.exchange = "IDEALPRO"
    bars = ib.reqHistoricalData(
            contract,
            endDateTime='',
            durationStr='2 D',
            barSizeSetting='1 min',
            whatToShow='BID',
            useRTH=True,
            formatDate=1)
    df = util.df(bars)
    ema = df["close"].ewm(span=50).mean().iloc[-1]
    emaG = ema
    openBid = pd.to_numeric(df['open'].iloc[-1])
    closeBid = pd.to_numeric(df['close'].iloc[-1])
    positions = ib.positions()
    action1 = "DONOTHING"
    if(checkOpenSell(symbol,openBid,closeBid,ema,currency,positions)):
        action1 = "OPENSELL"
        return action1
    elif(checkOpenBuy(symbol,openBid,closeBid,ema,currency,positions)):
        action1 = "OPENBUY"
        return action1
    elif(checkCloseSell(symbol,closeBid,ema,currency,positions)):
        action1 =  "CLOSESELL"
        return action1
    elif(checkCloseBuy(symbol,closeBid,ema,currency,positions)):
        action1 = "CLOSEBUY"
        return action1
    else:action1 = "DONOTHING"
    return action1

    

def validateAction():
    global long
    global short
    global action
    global sym
    global cur
    global sl
    newaction = 'a'
    newaction = getAction(sym,cur)
    print(newaction,": new action")
    if(not sl):
        if(newaction == "OPENSELL" and not short):
            action = newaction
        elif(newaction == "OPENBUY" and not long):
            action = newaction
        elif(newaction == "CLOSESELL" and short):
            action = newaction
        elif(newaction == "CLOSEBUY" and long):
            action = newaction
    

In [ ]:
#run all the time
def checkAction():
    global emaG
    global action
    global price
    global long
    global short
    global openprice
    global sl
    global slopenprice
    contract = Forex(tik)
    points = 0.0003
    stopLoss(openprice,price)
    stopLossExt(price)
    if(action == "OPENSELL" and price > emaG+points):
        print("sell order")
        order = MarketOrder('SELL', 100000)
        ib.placeOrder(contract, order)
        action = "CLOSESELL"
        short = True
        long = False
        openprice = price
    elif(action == "OPENBUY" and price < emaG-points):
        print("buy order")
        order = MarketOrder('BUY', 100000)
        ib.placeOrder(contract, order)
        action = "CLOSEBUY"
        short = False
        long = True
        openprice = price
    elif(action == "CLOSESELL" and price < emaG-points):
        print("buy order")
        order = MarketOrder('BUY', 200000)
        ib.placeOrder(contract, order)
        action = "CLOSEBUY"
        short = False
        long = True
        openprice = price
    elif(action == "CLOSEBUY" and price > emaG+points):
        print("sell order")
        order = MarketOrder('SELL', 200000)
        ib.placeOrder(contract, order)
        action = "CLOSESELL"
        short = True
        long = False
        openprice = price
    elif(sl and long and price <= emaG):
        order = MarketOrder('SELL', 100000)
        ib.placeOrder(contract, order)
        sl = False
        short = False
        long = False
        action = "DONOTHING"
        openprice = 0
        slopenprice = 0
    elif(sl and short and price >= emaG):
        order = MarketOrder('BUY', 100000)
        ib.placeOrder(contract, order)
        sl = False
        short = False
        long = False
        action = "DONOTHING"
        openprice = 0
        slopenprice = 0
    

In [ ]:
def updatePrice(tickers):
    global price
    global tik
    for t in tickers:
        if(t.contract.pair()==tik):
            price = t.ask
          
ib.pendingTickersEvent += updatePrice

In [ ]:
async def mainLoop():
    while True:
        await ib.updateEvent
        global price
        global eamG
        global action
        global firstT
        timeC = dt.datetime.now()
        min = timeC.minute 
        sec = timeC.second
        checkAction()
        if(min%1 == 0 and sec == 0):
            validateAction()
            print(price, " :current price")
            print(emaG, " :current EMA")
            print(str(action), ":current action")
            print("--------------") 
            firstT = False
            ib.sleep(1)
        

In [ ]:
loop = asyncio.get_event_loop()
loop.create_task(mainLoop())
loop.run_forever()

In [ ]:
#ib.pendingTickersEvent -= updatePrice

In [ ]:
#ib.disconnect()